In [1]:
import torch
import torch.nn as nn
torch.cuda.get_device_name (0)

'Tesla T4'

In [2]:
!cp -r "/content/gdrive/My Drive/space_Colab/datasets/yolo_crownHuman_total_in_a_folder" /content/dataset
!ls /content/dataset

cp: cannot stat '/content/gdrive/My Drive/space_Colab/datasets/yolo_crownHuman_total_in_a_folder': No such file or directory
ls: cannot access '/content/dataset': No such file or directory


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!mkdir -p /WORK1
!rm -rf /WORK1
!ln -s "/content/gdrive/My Drive/space_Colab" /WORK1


Mounted at /content/gdrive


In [9]:
import glob
import os
import shutil
from google.colab import drive # Added to ensure drive is mounted if not already

img_count_total = 40000
source_image_type = ".jpg"
source_yololabel_type = ".txt"
file_count_in_folder = 1000

source_dataset = "/DATA1/Datasets_mine/labeled/crowd_human_dataset/yolo2/yolo"
target_dataset = "/WORK1/dataset/CrowdHuman_YOLO_10_folders"

# --- Start of suggested fix for /WORK1 symlink issue ---
gdrive_source_path = "/content/gdrive/My Drive/space_Colab"
symlink_path = "/WORK1"

# Ensure Google Drive is mounted
if not os.path.exists("/content/gdrive/My Drive"):
    print("Google Drive not mounted. Mounting now...")
    drive.mount('/content/gdrive', force_remount=True)
    if not os.path.exists("/content/gdrive/My Drive"):
        print("Failed to mount Google Drive. Please check your connection or permissions.")
        # If mounting fails, the script cannot proceed with GDrive dependent operations.
        # Raising an error to stop execution.
        raise RuntimeError("Google Drive is not accessible.")

# Check if the Google Drive source path exists
if not os.path.exists(gdrive_source_path):
    print(f"Error: Google Drive path '{gdrive_source_path}' does not exist. Please ensure this folder exists in 'My Drive'.")
    # This path is critical for the symlink, so raise an error.
    raise FileNotFoundError(f"Required Google Drive folder '{gdrive_source_path}' not found.")

# Check if /WORK1 is a valid symlink to the GDrive path, or a directory
# If it's not a symlink, or it's a symlink but points to the wrong place, or it's missing
if not os.path.exists(symlink_path) or not os.path.islink(symlink_path) or os.readlink(symlink_path) != gdrive_source_path:
    print(f"Symlink {symlink_path} is missing, incorrect, or not pointing to {gdrive_source_path}. Attempting to re-create.")
    # Remove existing symlink or directory at /WORK1 if it exists
    if os.path.islink(symlink_path) or os.path.isdir(symlink_path):
        print(f"Removing existing {symlink_path}...")
        !rm -rf {symlink_path}
    # Create the symlink
    print(f"Creating symlink: {symlink_path} -> {gdrive_source_path}")
    !ln -s "{gdrive_source_path}" "{symlink_path}"

    # Verify again
    if not os.path.exists(symlink_path):
        print(f"ERROR: Failed to establish symlink {symlink_path}. Cannot proceed with dataset creation.")
        raise FileNotFoundError(f"Symlink {symlink_path} is not properly set up, and it's required for {target_dataset}")
    # os.path.isdir(symlink_path) will be true if the symlink points to an existing directory
    elif not os.path.isdir(symlink_path):
        print(f"ERROR: Symlink {symlink_path} points to a non-existent or non-directory path: {gdrive_source_path}. Please create '{gdrive_source_path}' in your Google Drive.")
        raise FileNotFoundError(f"Target of symlink {symlink_path} is invalid: {gdrive_source_path}")

# --- End of suggested fix for /WORK1 symlink issue ---


if not os.path.exists(target_dataset):
    # This os.makedirs call needs the parent directories to exist.
    # If WORK1 is now properly set up, this should work.
    os.makedirs(target_dataset)

images = sorted(glob.glob(os.path.join(source_dataset, "*" + source_image_type)))

for loop_folder in range(int(img_count_total / file_count_in_folder) + 1):
    print(f"Loop count #{loop_folder}")

    folder_path = os.path.join(target_dataset, f"folder_{loop_folder:02d}")
    os.makedirs(folder_path, exist_ok=True)

    start = loop_folder * file_count_in_folder
    end = start + file_count_in_folder

    for img_path in images[start:end]:
        base = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(source_dataset, base + source_yololabel_type)

        shutil.copy(img_path, folder_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, folder_path)


Loop count #0
Loop count #1
Loop count #2
Loop count #3
Loop count #4
Loop count #5
Loop count #6
Loop count #7
Loop count #8
Loop count #9
Loop count #10
Loop count #11
Loop count #12
Loop count #13
Loop count #14
Loop count #15
Loop count #16
Loop count #17
Loop count #18
Loop count #19
Loop count #20
Loop count #21
Loop count #22
Loop count #23
Loop count #24
Loop count #25
Loop count #26
Loop count #27
Loop count #28
Loop count #29
Loop count #30
Loop count #31
Loop count #32
Loop count #33
Loop count #34
Loop count #35
Loop count #36
Loop count #37
Loop count #38
Loop count #39
Loop count #40


In [11]:
import os
import shutil

# Assuming 'images' and 'loop_folder' are defined from previous cells.
# This loop provides the context for 'file' (as 'img_path') and 'i'.

# Note: The 'images' list is currently empty, meaning this loop will not execute
# any file moving operations. Please ensure 'source_dataset' in cell TxPPN1AAMXPR
# points to a directory containing image files.

for i, img_path in enumerate(images):
    filename = os.path.basename(img_path)
    file_mainname, file_extension = os.path.splitext(filename)

    source_img_file = os.path.join(source_dataset, filename)
    source_txt_file = os.path.join(source_dataset, file_mainname + source_yololabel_type)

    # new_folder uses the same format as 'folder_path' in cell TxPPN1AAMXPR
    new_folder = os.path.join(target_dataset, f"folder_{loop_folder:02d}")
    os.makedirs(new_folder, exist_ok=True)

    target_img_file = os.path.join(new_folder, filename)
    target_txt_file = os.path.join(new_folder, file_mainname + source_yololabel_type)

    try:
        print(f"#{loop_folder}/{i} move {filename}, {file_mainname + source_yololabel_type}...")
        if os.path.exists(source_img_file):
            shutil.move(source_img_file, target_img_file)
        else:
            print(f"Warning: Source image file not found: {source_img_file}")
        if os.path.exists(source_txt_file):
            shutil.move(source_txt_file, target_txt_file)
        else:
            print(f"Warning: Source label file not found: {source_txt_file}")
    except Exception as e:
        print(f"#{loop_folder}/{i} move failed: {e}")

if not images:
    print("Note: The 'images' list is empty. No files were processed by this cell.")
    print("Please check the 'source_dataset' path in cell TxPPN1AAMXPR to ensure it contains image files.")


Note: The 'images' list is empty. No files were processed by this cell.
Please check the 'source_dataset' path in cell TxPPN1AAMXPR to ensure it contains image files.


In [12]:
import os
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

%cd "/content/gdrive/My Drive/space_Colab"


Mounted at /content/gdrive
/content/gdrive/My Drive/space_Colab


In [13]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [14]:
!ln -s ‘/content/gdrive/My Drive/space_Colab’ /WORK1

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected


In [15]:
!ls -la /WORK1/darknet.official/darknet

!chmod 755 /WORK1/darknet.official/darknet

!ls -la /WORK1/darknet.official/darknet

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot access '/WORK1/darknet.official/darknet': No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
chmod: cannot access '/WORK1/darknet.official/darknet': No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot access '/WORK1/darknet.official/darknet': No such file or directory


In [ ]:
import requests, os

weights_filename = "/WORK1/cfg_YOLO/Pretrained/yolov3.weights"

weights_url = "https://pjreddie.com/media/files/yolov3.weights"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3503747512.py", line 1, in <cell line: 0>
    import requests, os
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1322, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1262, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1532, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1504, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1483, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceba

In [ ]:
import os
import cv2
import requests
import matplotlib.pyplot as plt

%matplotlib inline

def imShow(path):
    image = cv2.imread(path)
    height, width = image.shape[:2]

    resized_image = cv2.resize(
        image, (3 * width, 3 * height), interpolation=cv2.INTER_CUBIC
    )

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()


if not os.path.exists(weights_filename):
    r = requests.get(weights_url, stream=True)
    with open(weights_filename, "wb") as file:
        for block in r.iter_content(chunk_size=1024):
            if block:
                file.write(block)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1064513189.py", line 2, in <cell line: 0>
    import cv2
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1322, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1262, in _find_spec
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_cv2.py", line 96, in find_spec
    if not loader.find_spec():
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py", line 44, in find_spec
    spec = meta_path.find_spec(self.fullname, self.path, self.target)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap_external>", line 1532, in find_spec
  File "<fr

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

!ln -s ‘/content/gdrive/My Drive/space_Colab’ /WORK1

!ls -la /WORK1/darknet.official/darknet

!chmod 755 /WORK1/darknet.official/darknet

%cd /WORK1/darknet.official

!./darknet detector train /WORK1/cfg_YOLO/cfg.crowdHuman_colab/obj.data /WORK1/cfg_YOLO/cfg.crowdHuman_colab/crowd_human_yolov3_colab.cfg /WORK1/cfg_YOLO/Pretrained/darknet53.conv.74

Mounted at /content/gdrive
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ln: failed to create symbolic link '/WORK1/My': File exists
ln: failed to create symbolic link '/WORK1/space_Colab’': File exists
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot access '/WORK1/darknet.official/darknet': No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
chmod: cannot access '/WORK1/darknet.official/darknet': No such file or directory


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-497668926.py", line 11, in <cell line: 0>
    get_ipython().run_line_magic('cd', '/WORK1/darknet.official')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Tra

In [16]:
!./darknet detector train /WORK1/cfg_YOLO/cfg.crowdHuman_colab/obj.data /WORK1/cfg_YOLO/cfg.crowdHuman_colab/crowd_human_yolov3_colab.cfg /WORK1/cfg_YOLO/cfg.crowdHuman_colab/weights/crowd_human_yolov3_colab.backup

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
/bin/bash: line 1: ./darknet: Transport endpoint is not connected
